In [8]:
import os, json

import pandas as pd
import duckdb
import swifter

In [ ]:
MAX_TEXT_LEN_ALLOWED = 10000

In [2]:
for i in range(10):
    cwd = os.getcwd()
    r, c = os.path.split(cwd)
    if c == 'repo':
        print(f'new working dir: {cwd}')
        break
    os.chdir(r)

new working dir: c:\Users\aknof\Documents\GT\CSE_6250_BD4H\Project\repo


In [3]:
data_file = os.path.join('data', 'NOTEEVENTS.csv')
df = pd.read_csv(data_file)
df.head()

C:\Users\aknof\AppData\Local\Temp\ipykernel_5116\876603029.py:2: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_file)


,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
0,174,22532,167853.0,2151-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2151-7-16**] Dischar...
1,175,13702,107527.0,2118-06-14,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2118-6-2**] Discharg...
2,176,13702,167118.0,2119-05-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2119-5-4**] D...
3,177,13702,196489.0,2124-08-18,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2124-7-21**] ...
4,178,26880,135453.0,2162-03-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2162-3-3**] D...


In [4]:
con = duckdb.connect()

In [5]:
df_input = con.execute("""
select ROW_ID 
,SUBJECT_ID
,0 as chunk
,len(TEXT) lt
,TEXT
from df
order by lt desc
""").df()
df_input.head()

,ROW_ID,SUBJECT_ID,chunk,lt,TEXT
0,53810,93321,0,55728,Admission Date: [**2128-3-24**] ...
1,37847,42842,0,53330,Admission Date: [**2199-10-2**] ...
2,23428,43126,0,53169,Admission Date: [**2124-3-1**] D...
3,25215,5962,0,52508,Admission Date: [**2195-9-24**] ...
4,1484,66807,0,50663,Admission Date: [**2199-5-5**] D...


In [8]:
con.execute("""
select count(*)
from df_input
where lt > 10000
""").df()

,count_star()
0,40006


In [6]:
sql_chunk = f"""
{{union}}
select row_id
, subject_id
, {{chunk_num}} as chunk_num
, substring(text, chunk_window * {{chunk_num}}, {MAX_TEXT_LEN_ALLOWED}) as chunk_text
from par
where num_ch > {{chunk_num}}
"""

In [29]:
con.execute(f"""
select row_id
, subject_id
, lt
, cast(lt / {MAX_TEXT_LEN_ALLOWED} as int) + 1 num_ch
, lt / cast(cast(lt / {MAX_TEXT_LEN_ALLOWED} as int) + 1 as int) chunk_window
, text
from df_input
order by lt desc
limit 10
""").df()

,ROW_ID,SUBJECT_ID,lt,num_ch,chunk_window,TEXT
0,53810,93321,55728,6,9288,Admission Date: [**2128-3-24**] ...
1,37847,42842,53330,6,8888,Admission Date: [**2199-10-2**] ...
2,23428,43126,53169,6,8861,Admission Date: [**2124-3-1**] D...
3,25215,5962,52508,6,8751,Admission Date: [**2195-9-24**] ...
4,1484,66807,50663,6,8443,Admission Date: [**2199-5-5**] D...
5,34099,28941,50035,6,8339,Admission Date: [**2144-7-16**] ...
6,48774,51821,49417,5,9883,Admission Date: [**2160-10-9**] ...
7,42609,45317,47791,5,9558,Admission Date: [**2118-4-18**] ...
8,20493,55186,47181,5,9436,Admission Date: [**2121-4-27**] ...
9,19420,91074,46786,5,9357,Admission Date: [**2192-2-22**] ...


In [7]:
df_split = con.execute(f"""
with par as (
select row_id
, subject_id
, lt
, cast(lt / 10000 as int) + 1 num_ch
, lt / cast(cast(lt / {MAX_TEXT_LEN_ALLOWED} as int) + 1 as int) chunk_window
, text
from df_input
)
{sql_chunk.format(union='', chunk_num=0)}
{sql_chunk.format(union='union all', chunk_num=1)}
{sql_chunk.format(union='union all', chunk_num=2)}
{sql_chunk.format(union='union all', chunk_num=3)}
{sql_chunk.format(union='union all', chunk_num=4)}
{sql_chunk.format(union='union all', chunk_num=5)}
{sql_chunk.format(union='union all', chunk_num=6)}

""").df()
df_split.head()

,ROW_ID,SUBJECT_ID,chunk_num,chunk_text
0,53810,93321,0,Admission Date: [**2128-3-24**] ...
1,37847,42842,0,Admission Date: [**2199-10-2**] ...
2,23428,43126,0,Admission Date: [**2124-3-1**] D...
3,25215,5962,0,Admission Date: [**2195-9-24**] ...
4,1484,66807,0,Admission Date: [**2199-5-5**] D...


In [9]:
df_split.count()

ROW_ID        2126293
SUBJECT_ID    2126293
chunk_num     2126293
chunk_text    2126293
dtype: int64

In [37]:
con.execute(f"""
select max(len(chunk_text)) ml from df_split
""").df()

,ml
0,10000


In [40]:
con.execute(f"""
select chunk_num
, count(*) cnt
, avg(len(chunk_text)) avg_len
, max(len(chunk_text)) max_len
from df_split
group by chunk_num
order by 1
""").df()

,chunk_num,cnt,avg_len,max_len
0,0,2083180,1756.202070,9999
1,1,40016,6664.603084,10000
2,2,2747,7967.333819,10000
3,3,297,8637.542088,10000
4,4,47,8793.234043,10000
5,5,6,8764.833333,9289


In [12]:
records = df_split.to_json(orient="records")
parsed = json.loads(records)
parsed[0]

{'ROW_ID': 53810,
 'SUBJECT_ID': 93321,
 'chunk_num': 0,
 'chunk_text': 'Admission Date:  [**2128-3-24**]              Discharge Date:   [**2128-10-17**]\n\nDate of Birth:  [**2066-4-14**]             Sex:   M\n\nService: MEDICINE\n\nAllergies:\nNo Known Allergies / Adverse Drug Reactions\n\nAttending:[**First Name3 (LF) 2712**]\nChief Complaint:\nFatigue/Shortness of Breath\n\nMajor Surgical or Invasive Procedure:\nRight PICC placement [**2128-3-25**]\nCentral line placement [**2128-3-25**]\nBone marrow biopsy [**2128-3-25**]\nLeft subclavian central line placement [**2128-3-26**]\nBone marrow biopsy [**2128-4-8**]\nLeft PICC placement [**2128-4-10**]\nLeft internal jugular central line placement [**2128-4-20**]\nBone marrow biopsy [**2128-5-7**]\nBone marrow biopsy [**2128-5-27**]\nBone marrow biopsy [**2128-6-9**]\nBronchoscopy [**2128-6-10**]\nBone Marrow Biopsy on [**2128-6-29**]\nPercutaneous cholecystostomy tube placement [**2128-8-9**]\nPICC placement on [**2128-8-14**]\n\n\nHi

In [14]:
len(parsed)

2126293

In [13]:
with open(os.path.join('data', 'consumed', 'metamap_input.json'), 'w') as outfile:
    json.dump(parsed, outfile)


fin

In [31]:
# def write_chunk(x):
#     # print(x)
#     row, sub, chunk, text = x
#     file_name = f'row-{row}_sub-{sub}_chunk-{chunk}_.txt'
#     file = os.path.join('data', 'metamap', file_name)
#     with open(file, 'w') as outfile:
#         outfile.write(text)

In [12]:
# df_created = pd.DataFrame({'file_name': os.listdir(os.path.join('data', 'metamap'))})
# df_created.head()

,file_name
0,row-1000086_sub-28402_chunk-0.text
1,row-10000_sub-86502_chunk-0.text
2,row-10003_sub-84749_chunk-0.text
3,row-10004_sub-6116_chunk-0.text
4,row-10008_sub-21734_chunk-0.text


In [18]:
# df_created['file_parts'] = df_created['file_name'].str.split('_')
# df_created.head()

,file_name,file_parts
0,row-1000086_sub-28402_chunk-0.text,"[row-1000086, sub-28402, chunk-0.text]"
1,row-10000_sub-86502_chunk-0.text,"[row-10000, sub-86502, chunk-0.text]"
2,row-10003_sub-84749_chunk-0.text,"[row-10003, sub-84749, chunk-0.text]"
3,row-10004_sub-6116_chunk-0.text,"[row-10004, sub-6116, chunk-0.text]"
4,row-10008_sub-21734_chunk-0.text,"[row-10008, sub-21734, chunk-0.text]"


In [25]:
# df_created['ROW_ID'] = df_created['file_parts'].swifter.apply(lambda x: x[0].split('-')[1])
# df_created.head()

Pandas Apply:   0%|          | 0/36787 [00:00<?, ?it/s]

,file_name,file_parts,ROW_ID
0,row-1000086_sub-28402_chunk-0.text,"[row-1000086, sub-28402, chunk-0.text]",1000086
1,row-10000_sub-86502_chunk-0.text,"[row-10000, sub-86502, chunk-0.text]",10000
2,row-10003_sub-84749_chunk-0.text,"[row-10003, sub-84749, chunk-0.text]",10003
3,row-10004_sub-6116_chunk-0.text,"[row-10004, sub-6116, chunk-0.text]",10004
4,row-10008_sub-21734_chunk-0.text,"[row-10008, sub-21734, chunk-0.text]",10008


In [28]:
# # df_created['SUBJECT_ID'] = 
# # df_created['file_parts'].swifter.apply(lambda x: x[1].split('-')[1])
# df_created['file_parts'].swifter.apply(lambda x: x[1].split('-')[1]).head()
# # df_created.head()

Pandas Apply:   0%|          | 0/36787 [00:00<?, ?it/s]

IndexError: list index out of range

In [ ]:
# df_created['chunk_num'] = df_created['file_parts'].swifter.apply(lambda x: x[2].split('-')[1][0])
# df_created.head()

In [32]:
# df_split[['ROW_ID', 'SUBJECT_ID', 'chunk_num', 'chunk_text']].swifter.apply(write_chunk, axis=1)

Pandas Apply:   0%|          | 0/2126293 [00:00<?, ?it/s]

KeyboardInterrupt: 